In [14]:
from keras.preprocessing.text import Tokenizer

docs = ['너무 재밌어요', '참 최고에요', '참 잘 만든 영화에요',
        '추천하고 싶은 영화입니다', '한 번 더 보고 싶네요', '글쎄요',
        '별로에요', '생각보다 지루해요',  '연기가 어색해요',
        '재미없어요', '너무 재미없다', '참 재밌네요']

text_tokener = Tokenizer() 

# 1 fit_on_texts
text_tokener.fit_on_texts(docs)

# 1_1. word_index
text_tokener.word_index

# 1_2) texts_to_sequences 각 문장을 빈도순으로 숫자로 바꿔서 나타내줌
x = text_tokener.texts_to_sequences(docs)
x

from keras.preprocessing.sequence import pad_sequences # 패딩으로 와꾸 맞춰주기

pad_x = pad_sequences(x, padding='post', value=0)
pad_x 
# 알아서 최대 자릿수로 패딩을 채워준다.

pad_x = pad_sequences(x, padding='pre', value=0) # 근데 1부터는 값(value)를 가지고 있으니까.. 0으로 해주는게 좋겠다
pad_x

In [28]:
# 데이터 정리
import numpy as np
print(pad_x)
labels = np.array([1,1,1,1,1,0,0,0,0,0,0,1])
labels

[[ 0  0  0  2  3]
 [ 0  0  0  1  4]
 [ 0  1  5  6  7]
 [ 0  0  8  9 10]
 [11 12 13 14 15]
 [ 0  0  0  0 16]
 [ 0  0  0  0 17]
 [ 0  0  0 18 19]
 [ 0  0  0 20 21]
 [ 0  0  0  0 22]
 [ 0  0  0  2 23]
 [ 0  0  0  1 24]]


array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1])

# 5개로 이루어진(패딩상태) 12개의 문장과 그에 따른 1, 0 매칭 모델

In [37]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, LSTM, Conv1D

In [31]:
print(pad_x.shape)
print(labels.shape)

(12, 5)
(12,)


## 1) Embedding, Flatten, Dense 조합 

In [32]:
model = Sequential()
model.add(Embedding(25, 100, input_length=5))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5, 100)            2500      
_________________________________________________________________
flatten_1 (Flatten)          (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 501       
Total params: 3,001
Trainable params: 3,001
Non-trainable params: 0
_________________________________________________________________


## 2) Embedding, LSTM, Dense 조합 

In [36]:
model = Sequential()
model.add(Embedding(25, 100))
model.add(LSTM(20))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 100)         2500      
_________________________________________________________________
lstm_1 (LSTM)                (None, 20)                9680      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 12,201
Trainable params: 12,201
Non-trainable params: 0
_________________________________________________________________


## 3) Embedding, Conv1D, Dense 조합 

In [41]:
model = Sequential()
model.add(Embedding(25, 100, input_length=5))
model.add(Conv1D(20,2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 5, 100)            2500      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 4, 20)             4020      
_________________________________________________________________
flatten_4 (Flatten)          (None, 80)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 81        
Total params: 6,601
Trainable params: 6,601
Non-trainable params: 0
_________________________________________________________________


# 학습

In [42]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['acc'])
model.fit(pad_x, labels, epochs=30)
acc = model.evaluate(pad_x,labels)[1]
print('acc :',acc)

C:\Users\bitcamp\anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/30
12/12 [==============================] - 2s 127ms/step - loss: 0.6975 - acc: 0.3333
Epoch 2/30
12/12 [==============================] - 0s 249us/step - loss: 0.6844 - acc: 0.5833
Epoch 3/30
12/12 [==============================] - 0s 249us/step - loss: 0.6715 - acc: 0.7500
Epoch 4/30
12/12 [==============================] - 0s 331us/step - loss: 0.6587 - acc: 0.7500
Epoch 5/30
12/12 [==============================] - 0s 249us/step - loss: 0.6461 - acc: 0.8333
Epoch 6/30
12/12 [==============================] - 0s 166us/step - loss: 0.6336 - acc: 0.8333
Epoch 7/30
12/12 [==============================] - 0s 247us/step - loss: 0.6210 - acc: 0.9167
Epoch 8/30
12/12 [==============================] - 0s 250us/step - loss: 0.6084 - acc: 0.9167
Epoch 9/30
12/12 [==============================] - 0s 252us/step - loss: 0.5957 - acc: 0.9167
Epoch 10/30
12/12 [==============================] - 0s 166us/step - loss: 0.5828 - acc: 1.0000
Epoch 11/30
12/12 [==============================

# 결론 : 범주화 갯수, ouput node 갯수, input와꾸(띄어쓰기 갯수) 명시가 embedding의 순서고 범주화 갯수와 띄어쓰기 갯수 (즉 list shape가 두 차원으로 움직임) 를 이해하는게 제일 중요한 부분임